# Univariate Sort
1. Load the monthly CRSP file, the Fama-French Factors, and the estimated betas from the tidy_finance_*.sqlite database.

2. Create portfolio sorts based on the lagged beta. Specifically, you compute the breakpoint as the median lag beta each month. Then, you compute the returns of a portfolio that invests only in the stocks with a higher beta than the breakpoint and a portfolio that invests only in the stocks with a lower beta than the breakpoints. The portfolio weights can either be equal or value-weighted.

3. What are the monthly excess returns of both portfolios?

4. Does a portfolio that goes long high beta stocks and short low beta stocks yield an excess return significantly different from zero?

5. Write a general function for portfolio sorts based on a variable number of breakpoints. Then, compute portfolio returns based on lagged beta decile sorts.

6. What is the CAPM alpha of the ten portfolio returns? Is this finding in line with your expectations based on the CAPM implications?

7. Does a high beta minus low beta portfolio yield abnormal excess returns?

In [ ]:
import pandas as pd, numpy as np
import sqlite3, time
from copy import deepcopy
from plotnine import *
from mizani.formatters import percent_format, number_bytes_format
import statsmodels.api as sm
tidy_finance_python = sqlite3.connect( database="/Users/asbjornfyhn/Desktop/Emp Fin/data/tidy_finance_python.sqlite")
pd.read_sql_query('''SELECT name FROM sqlite_schema WHERE type ='table' AND  name NOT LIKE 'sqlite_%';''', con =tidy_finance_python)

### 1. Part
**Load the monthly CRSP file, the Fama-French Factors, and the estimated betas from the tidy_finance_*.sqlite database.**

Check all columns for all three tables 

In [ ]:
print(pd.read_sql_query('''SELECT * FROM crsp_monthly limit 1 ;''', con =tidy_finance_python).columns)
print(pd.read_sql_query('''SELECT * FROM factors_ff3_monthly limit 1 ;''', con =tidy_finance_python).columns)
print(pd.read_sql_query('''SELECT * FROM beta limit 1 ;''', con =tidy_finance_python).columns)

Read data from allt three tables in the database

In [98]:
crps_monthly = (pd.read_sql_query('select * from crsp_monthly;',con=tidy_finance_python, parse_dates={"month"}))
ff3_monthly = (pd.read_sql_query('select * from factors_ff3_monthly;',con=tidy_finance_python, parse_dates={"month"}))
beta_monthly = (pd.read_sql_query('select * from beta;',con=tidy_finance_python, parse_dates={"month"}))


Merge crps_monthly and beta_monthly on permno (company specific) and month (date). Then merge the result with fama_french_monthly on date.

In [ ]:
# merge crsp and beta on month and permno
df = pd.merge(crps_monthly, beta_monthly, on=["month",'permno'],how='left')
# merge with ff3 factors on month
df = pd.merge(df, ff3_monthly, on=["month"],how='left')
# drop rows where beta is missing values -- before 1964 
df.dropna(subset=['beta_monthly'], inplace=True)
# sort by month and permno
df = df.sort_values(by=['month','permno']).reset_index(drop=True)

In [ ]:
print(df[['month','permno','ret','ret_excess','beta_monthly']].iloc[:10,:])

### 2. Part and 3. Part
**2. Create portfolio sorts based on the lagged beta. Specifically, you compute the breakpoint as the median lag beta each month. Then, you compute the returns of a portfolio that invests only in the stocks with a higher beta than the breakpoint and a portfolio that invests only in the stocks with a lower beta than the breakpoints. The portfolio weights can either be equal or value-weighted.**

**3. What are the monthly excess returns of both portfolios?**

*First of we lag the monthly beta estimates by 1 month.*

*Next, we calculate the median beta and use it as a breaking point.*

*Lastly, we create two portfolios with equal weigths: one with all the companies with a beta below the median and one with betas above the median*

In [ ]:
# lag the beta by one month
df['beta_monthly'] = df.groupby('permno')['beta_monthly'].shift(1)
# calculate median of beta for each month
beta_median = df.groupby('month')['beta_monthly'].median()
# create a new column that indicates if the beta is above the median
df['beta_above_median'] = df['beta_monthly'] > df['month'].map(beta_median)
# for each month calculate the portfolio weight as equal weighted only for the stocks with beta above the median
df['portfolio_weight_above'] = df['beta_above_median'] / df.groupby('month')['beta_above_median'].transform('sum')
df['portfolio_weight_below'] = (1-df['beta_above_median']) / df.groupby('month')['beta_above_median'].transform('sum')

*We loop through all months in the dataframe. We check if the portfolio weights related to the month of inspection sum to 1. Then, we create the return for each portfolio and store it in the dataframe port_df*

In [ ]:
# create a dataframe with 'month' as index
port_df = pd.DataFrame(index=df['month'].unique(),columns=['portfolio_return_above', 'portfolio_return_below'])

for month in df['month'].unique():
    sum_ = df[df['month'] == month]['portfolio_weight_above'].sum()
    if not np.isclose(sum_, 1):
        print(f'The sum of portfolio weights is not 1 but {sum_:0.2f} for month {month}')
    else: 
        weights_below = df.loc[df['month'] == month, 'portfolio_weight_below'].values
        weights_above = df.loc[df['month'] == month, 'portfolio_weight_above'].values
        returns = df.loc[df['month'] == month, 'ret'].values
        excess_return = df.loc[df['month'] == month, 'ret_excess'].values
        port_df.loc[month, 'portfolio_return_below'] = weights_below @ returns.T
        port_df.loc[month, 'portfolio_excess_return_below'] = weights_below @ excess_return.T
        port_df.loc[month, 'portfolio_return_above'] = weights_above @ returns.T
        port_df.loc[month, 'portfolio_excess_return_above'] = weights_above @ excess_return.T
        
        # print(f'The portfolio return for month {month} is {weights_above @ returns.T:0.2%}')


*Transforming the dataframe to make it have the right formats etc.*
From the summary statistics, we see that the mean for the portoflio

In [ ]:
port_df = (port_df
           .reset_index(drop=False)
           .rename(columns={'index':'month'})
           .assign(month = lambda x: pd.to_datetime(x['month']))
           .sort_values(by='month')
           .set_index('month')
           .assign(portfolio_return_above = lambda x: x['portfolio_return_above'].astype(float),
                     portfolio_return_below = lambda x: x['portfolio_return_below'].astype(float),
                     portfolio_excess_return_above = lambda x: x['portfolio_excess_return_above'].astype(float),
                        portfolio_excess_return_below = lambda x: x['portfolio_excess_return_below'].astype(float))
           .rename(columns={
               'portfolio_return_above':'High beta Return',
               'portfolio_return_below':'Low beta Return',
               'portfolio_excess_return_above':'High beta Excess Return',
               'portfolio_excess_return_below':'Low beta Excess Return'
           })
           )
print(port_df.describe())

In [ ]:
# stack the dataframe
port_df_stack = (port_df
                 .stack()
                 .reset_index()
                 .rename(columns={'level_1':'Portfolio',0:'return'})
                 .assign(excess_indicator = lambda x: x['Portfolio'].str.contains('Excess'))
                 )
print(port_df_stack.head())
# create a boxplot of the portfolio returns
(ggplot(port_df_stack.dropna(subset=['return'])) +
    aes(y='return',x='Portfolio') +
    geom_boxplot(colour="#1F3552", 
                 fill="lightskyblue",
                 alpha=0.7, # transparency for the fill
                 outlier_shape=".", # shape of outlier
                 outlier_colour="steelblue", # colour of outlier
                 width=0.6
                 ) +
    geom_jitter(
        colour="grey",
        fill="black",
        width=.2,
        alpha=.05,
    ) + 
    theme_minimal(
        base_size=12,
        base_family='serif',
    ) + 
    labs(title='Portfolio returns for stocks with beta above median', x='Portfolio', y='Monthly Return') +
    scale_y_continuous(breaks=np.arange(-0.45, 0.401, 0.05), limits=[port_df_stack['return'].min(), port_df_stack['return'].max()]) +
    theme(axis_text_x=element_text(angle=45, hjust=1))
    )


### 4. Part
**Does a portfolio that goes long high beta stocks and short low beta stocks yield an excess return significantly different from zero?**

We can test the hypothesis that the excess return is zero using a t-test. 

Thus, we start by creating a portfolio that goes long in high beta stocks and short in low beta stocks. We then calculate the excess return of the portfolio.

In [ ]:
a = (port_df
     .dropna()
     .assign(long_short = lambda x: x['High beta Excess Return'] - x['Low beta Excess Return']))
print(a.head())

In [ ]:
model = (sm.OLS(exog=np.ones(len(a['long_short'])),endog=a['long_short']))
(model
 .fit(
     cov_type="HAC",
     cov_kwds={"maxlags": 6})
     .summary(
         xname='1',
         yname='Excess Return',
         alpha=0.05,
         title='Regression of High minus Low beta portfolio excess return on constant',
         slim=True)
     )

### 5. Part

**5. Write a general function for portfolio sorts based on a variable number of breakpoints. Then, compute portfolio returns based on lagged beta decile sorts.**

We create a portfolio portfolio_sort that creates portfolio sorts based on a variable number of breakpoints. After that it creates a portfolio for each month in the dataframe. The portfolio is simply created as a equal weighted portfolio of the stocks that are in the decile.

In [ ]:
def portfolio_sorts(
        df:pd.DataFrame,
        no_of_break_points:int=2,
        beta_col:str='beta_monthly',
        month_col:str='month',
        beta_lagged:bool=True,
        create_pf_weights:bool=False
        ):
    """ 
    This function performs the portfolio sorts and returns the portfolio returns
    """
    
    return_cols = ['month','ret','ret_excess','beta_quantile'] # define the columns to return

    df = df.dropna(subset=[beta_col]) # drop rows where beta is missing values
    
    # in case the beta is not lagged, we need to lag the beta by one month
    if beta_lagged: 
        df[beta_col] = df.groupby('permno')[beta_col].shift(1) # lag the beta by one month
    
    # calculate the break points for the beta
    v =  df.groupby(month_col).apply(lambda x: pd.qcut(x[beta_col]+1, no_of_break_points, labels=False))
    v = v.reset_index(drop=False).rename(columns={'level_1':'index',beta_col:'beta_quantile'}).set_index('index').drop(columns=month_col)
    df = pd.concat([df, v], axis=1, join='outer')

    # create portfolio weights
    if create_pf_weights:
        ts_sum = 0
        print(f'Creating portfolio weights for {no_of_break_points} portfolios and {len(df[month_col].unique())} months.')
        for i in range(no_of_break_points):
            ts = time.time()
            b = df['beta_quantile'] == i
            df[f'portfolio_weight_{i}'] = 0.0
            df.loc[b, f'portfolio_weight_{i}'] = 1.0
            return_cols.append(f'portfolio_weight_{i}') # appending the portfolio_weights columns to the return_cols

            
            for month in df[month_col].unique():
                m_cons = df[month_col] == month
                sum_ = df[m_cons][f'portfolio_weight_{i}'].sum()
                df.loc[(m_cons)&(b), f'portfolio_weight_{i}'] = 1/sum_
            
            ts_sum += time.time() - ts
            del ts
        print(f'Average time to calculate portfolio weights: {ts_sum/no_of_break_points:0.2f} seconds')
    return df[return_cols]


We use the function to create a portfolio for each decile.

In [ ]:
v = portfolio_sorts(deepcopy(df), 
                    no_of_break_points=10, 
                    beta_col='beta_monthly', 
                    month_col='month', 
                    beta_lagged=False,
                    create_pf_weights=True
                    )

Calculate the return for each portfolio.

In [ ]:
# calculate the returns for the portfolios
beta_sorted_df = pd.DataFrame(index=v['month'].unique(),columns=[f'portfolio_{i}' for i in range(10)])
for i in range(10):
    print(f'Calculating returns for portfolio {i}')
    beta_sorted_df.loc[:,f'portfolio_{i}'] = (v.assign(
        ret_con_i = lambda x: x['ret']*x[f'portfolio_weight_{str(i)}'],
        ).groupby('month')['ret_con_i'].sum())

In [ ]:
# https://ucsbcarpentry.github.io/2021-08-23-ucsb-python-online/07-visualization-ggplot-python/index.html
beta_sorted_df_mean = beta_sorted_df.mean().to_frame().reset_index(drop=False,names=['portfolio']).rename(columns={0:'avg_ret'})
beta_sorted_df_mean['avg_ret'] = beta_sorted_df_mean['avg_ret'].astype(float)
# create a chart with average returns for the portfolios
print(ggplot(beta_sorted_df_mean) +
    aes(x='portfolio',y='avg_ret') +
    geom_col() +
    labs(title='Average returns for beta sorted portfolios', x='Month', y='Monthly Return') +
    theme(axis_text_x=element_text(angle=45, hjust=1)) +
    scale_y_continuous(labels=percent_format()) + 
    theme_minimal(
        base_size=12,
        base_family='serif',
    ) +
    theme(axis_text_x=element_text(angle=45, hjust=1)) 
    )
del beta_sorted_df_mean

### 6. Part
**What is the CAPM alpha of the ten portfolio returns? Is this finding in line with your expectations based on the CAPM implications?**

We

In [ ]:
beta_melted_df = beta_sorted_df.reset_index(drop=False, names=['month']).melt(id_vars='month', value_vars=[f'portfolio_{i}' for i in range(10)], var_name='portfolio', value_name='return')

### 7. Part
**Does a high beta minus low beta portfolio yield abnormal excess returns?**